In [ ]:
pip install orion-ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires numpy>=1.18.5, but you have numpy 1.16.6 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.5 which is incompatible.
google-colab 1.0.0 requires pandas>=1.1.0, but you have pandas 0.24.2 which is incompatible.


In [ ]:
from orion import Orion
import yfinance as yf
import pandas as pd
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
#Importig Apple Stock data from yfinance
start = "2015-01-01"
end = "2021-01-01"
appl_df = yf.download("AAPL", start = start, end = end)
print(appl_df)
#Getting Date and Adjusted Closing Price of Apple Stock
appl_df_1 = appl_df.reset_index()[['Date','Adj Close']]
aapl = yf.Ticker('AAPL')


#Making table with Date converted to timestamp in nanoseconds
tmdata = pd.Series(appl_df_1['Date'].values.astype(np.int64))

#Merging table with timestamp with original Table
appl_df_2 = appl_df_1.merge(tmdata.rename('Timestamp'), how = 'inner', left_index= True, right_index = True)

#Dropping Date
appl_df_2.drop('Date', axis=1, inplace= True)

NameError: ignored

In [ ]:
hist = aapl.history(start=start, end=end)

In [ ]:
appl_df_3 = appl_df_2.reindex(columns = ['Timestamp', 'Adj Close'])
appl_final = appl_df_3.rename(columns={'Timestamp': 'timestamp', 'Adj Close': 'value'})
appl_final['value'] = appl_final['value'] /appl_final['value'].abs().max()
appl_final['timestamp'] = appl_final['timestamp'].div(1000000000)

In [ ]:
hyperparameters = {
    'keras.Sequential.LSTMTimeSeriesRegressor#1': {
        'epochs': 10,
        'verbose': True
    }
}

orion = Orion(
    pipeline='lstm_dynamic_threshold',
    hyperparameters=hyperparameters
)

In [ ]:
orion.fit(appl_final)

Train on 6808 samples, validate on 1703 samples
Epoch 1/10
6808/6808 [==============================] - 74s 11ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 2/10
6808/6808 [==============================] - 72s 11ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 3/10
6808/6808 [==============================] - 73s 11ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1150 - val_mse: 0.1150
Epoch 4/10
6808/6808 [==============================] - 73s 11ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1149 - val_mse: 0.1149
Epoch 5/10
6808/6808 [==============================] - 73s 11ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 6/10
6808/6808 [==============================] - 73s 11ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1126 - val_mse: 0.1126
Epoch 7/10
6808/6808 [==============================] - 72s 11ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1150 - val_mse: 0.1150


In [ ]:
anomalies = orion.detect(appl_final)

8511/8511 [==============================] - 24s 3ms/step


In [ ]:
anomalies['start'] = pd.to_datetime(anomalies['start'], unit='s')
anomalies['end'] = pd.to_datetime(anomalies['end'], unit='s')
anomalies

,start,end,severity
0,2019-12-07 12:00:00,2019-12-25 06:00:00,0.071621


In [ ]:
anomalies_hist = [aapl.history(start=anomalies['start'].iloc[i], end=anomalies['end'].iloc[i]) for i in range(anomalies.shape[0])]
print(anomalies_hist)

[                 Open       High        Low      Close     Volume  Dividends  \
Date                                                                           
2019-12-09  66.237922  66.434180  64.989216  65.482323  128042400          0   
2019-12-10  65.894489  66.255118  65.222292  65.865051   90420400          0   
2019-12-11  65.945998  66.507796  65.869947  66.426834   78756800          0   
2019-12-12  65.693303  66.865960  65.580456  66.596100  137310400          0   
2019-12-13  66.596106  67.538155  66.466083  67.501358  133587600          0   
2019-12-16  67.955217  68.885004  67.950313  68.656845  128186000          0   
2019-12-17  68.585698  69.125410  68.396793  68.791771  114158400          0   
2019-12-18  68.642115  69.157300  68.475295  68.627396  116028400          0   
2019-12-19  68.568524  68.980669  68.433598  68.696091   98369200          0   
2019-12-20  69.238263  69.341296  68.337915  68.553802  275978000          0   
2019-12-23  68.821220  69.733832  68.78

In [ ]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=hist.index,y=hist['Close'],name='Price'),secondary_y=False)
for i in range(len(anomalies_hist)):
  fig.add_trace(go.Scatter(x=anomalies_hist[i].index.values,y=anomalies_hist[i]['Close'],name='Anomaly' + str(i)),secondary_y=False)
fig.show()